In [1]:
import pandas as pd
import os
import numpy as np

words = ["ameise", "begräbnis", "deichsel", "elster", "fledermaus", "gurke", "hagebutte", "hebamme", "kartoffel", "maulwurf", "pflaume", "stecknadel", "ziege", "zimmerfliege"]

In [2]:
def na_detector(df_spalte):
    df_spalte_neu = []
    for number in df_spalte:
        if number > 999:
            number = number % 999
        df_spalte_neu.append(number)
    return df_spalte_neu

In [3]:
for word in words:
    path = f'/home/kopatsch/Masterarbeit/MA/masterarbeit/data/lexem/{word}'
    entries = os.listdir(path)
    
    df_list = []

    for entry in entries:
        df = pd.read_csv(f"{path}/{entry}", sep = "\t")
        df_list.append(df)
    

    result = pd.concat(df_list)
    result = result.drop(columns = ["Unnamed: 0"])
    
    result = result.groupby("Koordinaten").agg(Fragebogen_maurer = ("Fragebogen_maurer", set),
                                                Fragebogen_DWA = ("Fragebogen_DWA", set),
                                                Ort = ("Ort", set),
                                                item_maurer = ("item_maurer", set),
                                                item_DWA = ("item_DWA", set),
                                                Levenshtein_item = ("Levenshtein_item", "sum"),
                                                phontype_maurer = ("phontype_maurer", set),
                                                phontype_DWA = ("phontype_DWA", set),
                                                Levenshtein_phontype = ("Levenshtein_phontype", "sum"),
                                                lextype_maurer = ("lextype_maurer", set),
                                                lextype_DWA = ("lextype_DWA", set),
                                                Levenshtein_lextype = ("Levenshtein_lextype", "sum"))
    

    item = na_detector(result["Levenshtein_item"])
    result["Levenshtein_item"] = item
    phontype = na_detector(result["Levenshtein_phontype"])
    result["Levenshtein_phontype"] = phontype
    lextype = na_detector(result["Levenshtein_lextype"])
    result["Levenshtein_lextype"] = lextype

    result_clean = pd.DataFrame()
    for column in result:
        liste = []
        for j in result[column]:
            j = str(j)
            j = j.replace("{", "")
            j = j.replace("}", "")
            j = j.replace("\'", "")
            j = j.replace("\"", "")
            j = j.replace("nan", "na")
            j = j.replace("na, ", "")
            j = j.replace(", na", "")
            j = j.strip()
            liste.append(j)
        result_clean[column] = liste

    result_clean.insert(loc=3, column='Koordinaten', value=result.index)
    long = []
    lat = []
    for coords in result_clean["Koordinaten"]:
        coords = str(coords)
        coord = coords.split(", ")
        long.append(float(coord[0]))
        lat.append(float(coord[1]))
    
    result_clean.insert(loc=4, column='Longitude', value=long)
    result_clean.insert(loc=5, column='Latitude', value=lat)

    result_clean.to_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/lexem_gesamt/all_levenshtein_lexem_added_{word}.csv", sep = "\t")

In [4]:
# Liste der Untersuchungsworte
# Vereinfacht die Automatisierungen verschidener Funktionen im weiteren Scriptverlauf
words = ["ameise", "begräbnis", "deichsel", "elster", "fledermaus", "gurke", "hagebutte", "hebamme", "kartoffel", "maulwurf", "pflaume", "stecknadel", "ziege", "zimmerfliege"]

levenshtein_item = []
levenshtein_phon= []
levenshtein_lex = []
for word in words:
    df = pd.read_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/levenshtein_single_files/{word}.csv", sep= "\t")
    df1 = df.drop(df[df["Levenshtein Item Min"] == 999].index)
    liste1 = list(df1["Levenshtein Item Min"])
    for x in liste1:
        levenshtein_item.append(x)
        levenshtein_phon.append(x)
        levenshtein_lex.append(x)
    df2 = df.drop(df[df["Levenshtein Phontype Min"] == 999].index)
    liste2 = list(df2["Levenshtein Phontype Min"])
    for x in liste2:
        levenshtein_phon.append(x)
        levenshtein_lex.append(x)
    df3 = df.drop(df[df["Levenshtein Lextype Min"] == 999].index)
    liste3 = list(df3["Levenshtein Lextype Min"])
    for x in liste3:
        levenshtein_lex.append(x)


mittel_item = np.mean(levenshtein_item)
stvar_item = np.std(levenshtein_item)
mittel_phon = np.mean(levenshtein_phon)
stvar_phon = np.std(levenshtein_phon)
mittel_lex = np.mean(levenshtein_lex)
stvar_lex = np.std(levenshtein_lex)

for word in words:
    df = pd.read_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/lexem_gesamt/all_levenshtein_lexem_added_{word}.csv", sep= "\t")
    df.insert(loc = 10, column = "z_score_item", value = ((df["Levenshtein_item"]-mittel_item)/stvar_item))
    df.insert(loc = 14, column = "z_score_phontype", value = ((df["Levenshtein_phontype"]-mittel_phon)/stvar_phon))
    df.insert(loc = 18, column = "z_score_lextype", value = ((df["Levenshtein_lextype"]-mittel_lex)/stvar_lex))
    df = df.drop(columns = ["Unnamed: 0"])
    df.to_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/lexem_gesamt/all_levenshtein_lexem_added_{word}.csv", sep= "\t")